In [1]:
import os

In [2]:
os.chdir('../../')

In [3]:
%pwd

'/Users/bhikipallai/Desktop/Projects/95Mobiles'

In [4]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class laptop_modelevalconfig:
    root_dir: Path
    X_train: Path
    X_test: Path
    y_train: Path
    y_test: Path
    save_score: Path
    model_for_train: Path

In [5]:
from src.laptop.constants import *
from src.laptop.utils.common import read_yaml,create_directory

[2024-10-05 22:24:28,411: INFO]: Logging file start


In [6]:
class ConfigurationManger:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,param_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)
        create_directory([self.config.data_root])
    
    def get_model_eval(self)->laptop_modelevalconfig:
        config = self.config.model_eval
        create_directory([config.root_dir])

        model_eval = laptop_modelevalconfig(
            root_dir=config.root_dir,
            X_train = config.X_train,
            X_test = config.X_test,
            y_train=config.y_train,
            y_test=config.y_test,
            save_score=config.save_score,
            model_for_train=config.model_for_train
        )

        return model_eval

In [7]:
from sklearn.metrics import r2_score
import pandas as pd
import mlflow
import dagshub
import joblib
import json

In [8]:
class ModelEval:
    def __init__(self,config:laptop_modelevalconfig):
        self.config = config
    
    def model_evl(self):
        X_train = pd.read_csv(self.config.X_train)
        X_test = pd.read_csv(self.config.X_test)
        y_train = pd.read_csv(self.config.y_train)
        y_test = pd.read_csv(self.config.y_test)
        
        model = joblib.load(self.config.model_for_train)

        dagshub.init(repo_owner='Vicky7873', repo_name='95Mobiles', mlflow=True)
        mlflow.set_registry_uri("https://dagshub.com/Vicky7873/95Mobiles.mlflow")
        mlflow.set_experiment("laptop model eval")

        with mlflow.start_run():
            train_acc = model.score(X_train,y_train)
            print(f"my training acc: {train_acc}")
            y_pred = model.predict(X_test)
            test_acc = r2_score(y_test,y_pred)
            print(f"my testing acc: {test_acc}")

            score = {
                "train score: ": train_acc,
                "test score: ": test_acc
            }

            mlflow.log_metric("training acc ",train_acc)
            mlflow.log_metric("test acc ",test_acc)

            with open(self.config.save_score,'w') as file:
                json.dump(score,file,indent=4)


In [9]:
try:
    config = ConfigurationManger()
    eval_config = config.get_model_eval()
    me = ModelEval(config=eval_config)
    me.model_evl()

except Exception as e:
    raise e


[2024-10-05 22:24:29,523: INFO]: yaml: config/laptop_config.yaml loaded successfully
[2024-10-05 22:24:29,524: INFO]: yaml: params/laptop_params.yaml loaded successfully
[2024-10-05 22:24:29,525: INFO]: created directory at: data
[2024-10-05 22:24:29,525: INFO]: created directory at: data/laptop/model_eval
[2024-10-05 22:24:29,994: INFO]: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"


Accessing as Vicky7873

[2024-10-05 22:24:30,003: INFO]: Accessing as Vicky7873
[2024-10-05 22:24:30,470: INFO]: HTTP Request: GET https://dagshub.com/api/v1/repos/Vicky7873/95Mobiles "HTTP/1.1 200 OK"


Initialized MLflow to track repo "Vicky7873/95Mobiles"

[2024-10-05 22:24:30,480: INFO]: Initialized MLflow to track repo "Vicky7873/95Mobiles"


Repository Vicky7873/95Mobiles initialized!

[2024-10-05 22:24:30,481: INFO]: Repository Vicky7873/95Mobiles initialized!
my training acc: 0.9723160905574255
my testing acc: 0.8002737253225795


2024/10/05 22:24:32 INFO mlflow.tracking._tracking_service.client: 🏃 View run fearless-stoat-206 at: https://dagshub.com/Vicky7873/95Mobiles.mlflow/#/experiments/6/runs/750e4d18f71e41bb8656f373a0b4458d.
2024/10/05 22:24:32 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Vicky7873/95Mobiles.mlflow/#/experiments/6.
